In [ ]:
import numpy as np
import pandas as pd
import re
import os

import scipy.stats as stats

#visualizing results
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
%matplotlib inline
import seaborn as sns
sns.set_context('poster')

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import silhouette_score, davies_bouldin_score, mean_squared_error, mean_absolute_error, r2_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC 
from sklearn.neighbors import KNeighborsClassifier 
import xgboost as xgb

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, roc_auc_score

import shap
# load JS visualization code to notebook
shap.initjs()

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

### Get data

In [ ]:
path_data_bl = '/Users/abbieschindler/Documents/ProgrammingFun/iPythonScripts/Clinial/data_bl.csv'
path_data_fu = '/Users/abbieschindler/Documents/ProgrammingFun/iPythonScripts/Clinial/data_fu.csv'

In [ ]:
data_bl = pd.read_csv(path_data_bl, encoding='latin1', index_col=0)
data_bl = pd.DataFrame(data=data_bl)
print(data_bl.shape)
print(data_bl['VisitID'].value_counts())
data_bl.head()

In [ ]:
data_fu = pd.read_csv(path_data_fu, encoding='latin1', index_col=0)
data_fu = pd.DataFrame(data=data_fu)
print(data_fu.shape)
print(data_fu['VisitID'].value_counts())
data_fu.head()

### create cluster df

In [ ]:
data_PCL = data_bl[['level_0', 'pcl5total']]
data_PCL['level_0fu'] = data_fu['level_0']
data_PCL['pcl5total_fu'] = data_fu['pcl5total']
data_PCL['pcl_diff'] = (data_PCL['pcl5total_fu'] - data_PCL['pcl5total'])
print(data_PCL.shape)
data_PCL = data_PCL.dropna()
print(data_PCL.shape)
data_PCL.head()

### determine cluster number

In [ ]:
data_PCLfeat = data_PCL[['pcl5total', 
                         'pcl5total_fu', 'pcl_diff']]
# center and scale the data
scaler = StandardScaler()
#scaler = RobustScaler()

data_PCLfeat_scaled = scaler.fit_transform(data_PCLfeat)

In [ ]:
data = data_PCLfeat_scaled

#pick cluster number based on silhouette coefficient
k_range = range(2,15)

base_scores = []
sil_scores = []
db_scores = []
ch_scores = []
mse_scores = []

for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=39)
    km_ss.fit(data)
    
    sil_scores.append(silhouette_score(data, km_ss.labels_))
    db_scores.append(davies_bouldin_score(data, km_ss.labels_))
    #ch_scores.append(calinski_harabasz_score(data, km_ss.labels_))
    mse_scores.append(km_ss.inertia_)
    
# plot the results
plt.plot(k_range, sil_scores)
plt.title('AUDIT-C Questions K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.show()

# plot the results
plt.plot(k_range, db_scores)
plt.title('AUDIT-C Questions K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Davies Bouldin Score')
plt.show()
    
# plot the results
plt.plot(k_range, mse_scores)
plt.title('AUDIT-C Questions K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
#viz dendrogram to find if three clusters makes sense
plt.figure(figsize=(10, 7))  
plt.title("Dendrogram")  
plt.ylabel("Distance (dissimilarity)")
plt.xlabel("Participants")
dend = shc.dendrogram(shc.linkage(data_PCLfeat_scaled, method='ward'), 
                      distance_sort='ascending',
                      show_leaf_counts=True, leaf_font_size=8)

In [ ]:
#viz clustering with heat map of AUDIT-C answers
sns.clustermap(data_PCLfeat_scaled, metric="euclidean", standard_scale=1, method="ward", cmap="Blues")

In [ ]:
#choose k=3 clusters and fit data
km_3 = KMeans(n_clusters=3,random_state=39)
km_3.fit(data_PCLfeat_scaled)
#add cluster info to df
data_PCL['kmeans_cluster'] = ["cluster_" + str(label) for label in km_3.labels_ ]
print(data_PCL.shape)
print(data_PCL['kmeans_cluster'].value_counts())
data_PCL.head(1)

In [ ]:
data_PCL['kmeans_cluster'].value_counts()

In [ ]:
cols = ['pcl5total', 'pcl5total_fu', 'pcl_diff']

for variable in cols:
    try:
        plt.figure(figsize=(7,7))
        g = sns.barplot(x='kmeans_cluster', y=variable, data=data_PCL, ci=68, palette="rocket", order=["cluster_0", "cluster_1", "cluster_2"])
        plt.show()
    except:
        pass

In [ ]:
data_PCL.head()

In [ ]:
melt = pd.melt(data_PCL, id_vars=['level_0', 'kmeans_cluster'], value_vars=['pcl5total', 'pcl5total_fu'],
              var_name='timepoint', value_name='PCL5total')
melt.head()

In [ ]:
g = sns.catplot(x="timepoint", y="PCL5total", hue="kmeans_cluster",
                capsize=.2, height=7, aspect=1,
                kind="point", data=melt, palette=['green', 'darkorange', 'red'])



### try to predict cluster assignment based on exposure at bl

In [ ]:
#add cluster info back to original df
cluster_dict = dict(zip(data_PCL['level_0'], data_PCL['kmeans_cluster']))
data_bl['cluster'] = data_bl['level_0'].map(cluster_dict)

#create int cluster column
data_PCL['kmeans_cluster_num'] = [int(x.split('_')[-1]) for x in data_PCL['kmeans_cluster']]
cluster_dict = dict(zip(data_PCL['level_0'], data_PCL['kmeans_cluster_num']))
data_bl['cluster_num'] = data_bl['level_0'].map(cluster_dict)
print(data_bl.shape)

In [ ]:
keep = ['level_0', 'VisitID', 'age', 'gender',
       'attendDem_HCW', 'lecAtotal', 
         'ceaAPi1', 'ceaAPi2',
       'ceaAPi3', 'ceaMC', 'ceaPriorTimeWeeks', 'ceaPTWi1', 'ceaPTWi2',
       'ceaPTWi3', 'ceaPTWi4', 'ceaPTWi5', 'ceaPTWi6', 'ceaPTWi7',
       'ceaPTWi8', 'ceaPTWi9', 'ceaPTWi10', 'ceaPTWi11', 'ceaPTWi12',
       'ceaPTWi13', 'phq9i1', 'phq9i2', 'phq9i3', 'phq9i4', 'phq9i5',
       'phq9i6', 'phq9i7', 'phq9i8', 'phq9i9', 'stni1', 'stni10a',
       'stni11a', 'stni2', 'stni3', 'stni4', 'stni5', 'stni6', 'stni7',
       'stni8', 'stni9', 'gad7i1', 'gad7i2', 'gad7i3', 'gad7i4', 'gad7i5',
       'gad7i6', 'gad7i7', 'isi1', 'isi2', 'isi3', 'isi4', 'isi5', 'isi6',
       'isi7', 'infield', 'work', 'cluster', 'cluster_num']

data_bl_dropna = data_bl[keep].dropna()
print(data_bl_dropna.shape)
data_bl_dropna.head()

In [ ]:
#create/map factors
gender_dict = {'F': 0, 'M': 1, 'NB': 2, 'O':3}
data_bl_dropna['gender'] = data_bl_dropna['gender'].map(gender_dict)
data_bl_dropna['gender'] = data_bl_dropna['gender'].astype(object)

HCW_dict = {True: 0, False: 1}
data_bl_dropna['attendDem_HCW'] = data_bl_dropna['attendDem_HCW'].map(HCW_dict)
data_bl_dropna['attendDem_HCW'] = data_bl_dropna['attendDem_HCW'].astype(object)

data_bl_dropna['cluster_num'] = data_bl_dropna['cluster_num'].astype('int')
#data_bl_dropna['cluster_num'] = data_bl_dropna['cluster_num'].astype(object)

print(data_bl_dropna.shape)
data_bl_dropna.head()

In [ ]:
features_exposure = ['age', 'gender', 'attendDem_HCW', 'lecAtotal', 
         'ceaAPi1', 'ceaAPi2',
       'ceaAPi3', 'ceaMC', 'ceaPriorTimeWeeks', 'ceaPTWi1', 'ceaPTWi2',
       'ceaPTWi3', 'ceaPTWi4', 'ceaPTWi5', 'ceaPTWi6', 'ceaPTWi7',
       'ceaPTWi8', 'ceaPTWi9', 'ceaPTWi10', 'ceaPTWi11', 'ceaPTWi12',
       'ceaPTWi13']

features_MH = ['age', 'gender', 'attendDem_HCW',
              'phq9i1', 'phq9i2', 'phq9i3', 'phq9i4', 'phq9i5',
       'phq9i6', 'phq9i7', 'phq9i8', 'phq9i9', 'stni1', 'stni10a',
       'stni11a', 'stni2', 'stni3', 'stni4', 'stni5', 'stni6', 'stni7',
       'stni8', 'stni9', 'gad7i1', 'gad7i2', 'gad7i3', 'gad7i4', 'gad7i5',
       'gad7i6', 'gad7i7', 'isi1', 'isi2', 'isi3', 'isi4', 'isi5', 'isi6',
       'isi7']

features_all = ['age', 'gender', 'attendDem_HCW', 'lecAtotal', 
         'ceaAPi1', 'ceaAPi2',
       'ceaAPi3', 'ceaMC', 'ceaPriorTimeWeeks', 'ceaPTWi1', 'ceaPTWi2',
       'ceaPTWi3', 'ceaPTWi4', 'ceaPTWi5', 'ceaPTWi6', 'ceaPTWi7',
       'ceaPTWi8', 'ceaPTWi9', 'ceaPTWi10', 'ceaPTWi11', 'ceaPTWi12',
       'ceaPTWi13', 'phq9i1', 'phq9i2', 'phq9i3', 'phq9i4', 'phq9i5',
       'phq9i6', 'phq9i7', 'phq9i8', 'phq9i9', 'stni1', 'stni10a',
       'stni11a', 'stni2', 'stni3', 'stni4', 'stni5', 'stni6', 'stni7',
       'stni8', 'stni9', 'gad7i1', 'gad7i2', 'gad7i3', 'gad7i4', 'gad7i5',
       'gad7i6', 'gad7i7', 'isi1', 'isi2', 'isi3', 'isi4', 'isi5', 'isi6',
       'isi7', 'infield', 'work']

features_best = ['age', 'ceaPriorTimeWeeks', 'infield', 'phq9i5', 'isi5', 'stni1', 'work', 'gad7i6', 'phq9i4', 'ceaPTWi10']
from sklearn.preprocessing import label_binarize
              
#split data
train, test = train_test_split(data_bl_dropna, test_size = .3, random_state=39, stratify = data_bl_dropna['cluster'])

Y_train = train['cluster_num']
Y_test = test['cluster_num']

#create feature sets
X_train_exp = train[features_exposure]
X_test_exp = test[features_exposure]

X_train_mh = train[features_MH]
X_test_mh = test[features_MH]

X_train_all = train[features_all]
X_test_all = test[features_all]

X_train_best = train[features_best]
X_test_best = test[features_best]

In [ ]:
#scale data algo
scaler = StandardScaler()

#k fold algo
strat_k_fold = StratifiedKFold(n_splits=10)

#classifier algos
dm_cv = DummyClassifier(strategy='stratified', random_state=39)
lr_cv = LogisticRegression(random_state=39, class_weight='balanced')
rf_cv = RandomForestClassifier(random_state=39, class_weight='balanced')
svm_cv = SVC(kernel='linear', probability=True, class_weight='balanced') 
knn_cv = KNeighborsClassifier()
gb_cv = GradientBoostingClassifier(random_state=39)
ab_cv = AdaBoostClassifier(random_state=39)

#dic with classifier and feature importance attribute name
models_dic = {'dm_cv': (dm_cv, 'none'), 
              'lr_cv': (lr_cv, 'coef'), 
              'rf_cv': (rf_cv, 'feature_importance'), 
              'svm_cv':(svm_cv, 'coef'), 
              'knn_cv': (knn_cv, 'none'), 
              'gb_cv': (gb_cv, 'feature_importance'),
              'ab_cv': (ab_cv, 'feature_importance')}

In [ ]:
def feature_importance(X, y, model_instance, feature_names, fi_name):
    #takes in features (X) and classess (y), model, column names for features in X, and name of attribute for feature importance
    #returns dictionary of feature names and coef/feature importance values
    
    feature_importance_dic = {}
    
    model_instance.fit(X, y)
    
    if fi_name == 'coef':
        coef = model_instance.coef_[0]
        feature_importance_dic = dict(zip(feature_names, coef))
    if fi_name == 'feature_importance':
        coef = model_instance.feature_importances_
        feature_importance_dic = dict(zip(feature_names, coef))
    if fi_name == 'none':
        coef = np.zeros(len(feature_names))
        feature_importance_dic = dict(zip(feature_names, coef))
    
    return feature_importance_dic

In [ ]:
def classification_pipeline(X, y, cv_instance, model_instance, feature_names, fi_name):
    
    #scale data
    data_scaled = scaler.fit_transform(X)
    
    #generate cross-val sets
    cv = list(cv_instance.split(X, y))
    
    #predict class and predict probability 
    y_pred = cross_val_predict(model_instance, X, y, cv=cv, method='predict')
    y_pred_prob = cross_val_predict(model_instance, X, y, cv=cv, method='predict_proba')
    
    #generate confusion matrix
    conf_mat = confusion_matrix(y, y_pred)
    print('Confusion matrix:', conf_mat)
    
    #generate ROC_AUC
    #ROC_AUC = metrics.roc_auc_score(y, y_pred_prob[:,1])
    #print("ROC_AUC: ", ROC_AUC)
    
    # generate additional metrics
    recall = metrics.recall_score(y,y_pred, average='weighted')
    precision = metrics.precision_score(y,y_pred, average='weighted')
    accuracy = metrics.accuracy_score(y,y_pred)
    F1 = metrics.f1_score(y,y_pred, average='weighted')
    print("Sensitivity/Recall (TPR): ",recall)
    print("Precision (PPV): ", precision)
    print("Accuracy: ", accuracy)
    print("F1:", F1)
    
    #determine feature importance
    feature_dic = feature_importance(X, y, model_instance, feature_names, fi_name)
    
    #create dic
    data_dic = {}
    data_dic['y_pred'] = y_pred
    data_dic['y_pred_prob'] = y_pred_prob
    data_dic['conf_mat'] = conf_mat
    #data_dic['ROC_AUC'] = ROC_AUC
    data_dic['recall'] = recall
    data_dic['precision'] = precision
    data_dic['accuracy'] = accuracy
    data_dic['F1'] = F1
    
    data_dic = {**data_dic, **feature_dic}
    
    return data_dic

In [ ]:
feature_set = 'exposure'
feature_names = features_exposure

data_features_exposure = {}

for name, model in models_dic.items():
    print(f'{name} model with {feature_set} features:')
    data_features_exposure[name + '_' + feature_set] = classification_pipeline(X_train_exp, Y_train, strat_k_fold, model[0], feature_names, model[1])
    print('\n')

In [ ]:
feature_set = 'MH'
feature_names = features_MH

data_features_MH = {}

for name, model in models_dic.items():
    print(f'{name} model with {feature_set} features:')
    data_features_MH[name + '_' + feature_set] = classification_pipeline(X_train_mh, Y_train, strat_k_fold, model[0], feature_names, model[1])
    print('\n')

In [ ]:
feature_set = 'all'
feature_names = features_all

data_features_all = {}

for name, model in models_dic.items():
    print(f'{name} model with {feature_set} features:')
    data_features_all[name + '_' + feature_set] = classification_pipeline(X_train_all, Y_train, strat_k_fold, model[0], feature_names, model[1])
    print('\n')

In [ ]:
feature_set = 'best'
feature_names = features_best

data_features_best = {}

for name, model in models_dic.items():
    print(f'{name} model with {feature_set} features:')
    data_features_best[name + '_' + feature_set] = classification_pipeline(X_train_best, Y_train, strat_k_fold, model[0], feature_names, model[1])
    print('\n')

In [ ]:
#put dics in pandas df 
final_dic = {**data_features_exposure, **data_features_MH, **data_features_all}
data_pandas = pd.DataFrame.from_dict(data = final_dic, orient='index').reset_index()
data_pandas.sort_values('F1', ascending=False).head()

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scipy import interp
from itertools import cycle

X=X_train_all
# Binarize the output
y = label_binarize(Y_train, classes=[0, 1, 2])
n_classes = y.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                    random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True,
                                 random_state=random_state))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


In [ ]:
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(20,10))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

lw=2
colors =['green', 'darkorange', 'red']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Support Vector Machine multi-class ROC (all features)')
plt.legend(loc="lower right")
plt.show()

### explain features

In [ ]:
# train a SVM classifier
#classifier algos
dm = DummyClassifier(strategy='stratified', random_state=39)
lr = LogisticRegression(random_state=39, class_weight='balanced')
rf = RandomForestClassifier(random_state=39, class_weight='balanced')
svm = SVC(kernel='linear', probability=True, class_weight='balanced') 
knn = KNeighborsClassifier()
gb = GradientBoostingClassifier(random_state=39)
ab = AdaBoostClassifier(random_state=39)

model = svm

model.fit(X_train_all, Y_train)


y_pred = model.predict(X_test_all)
y_predp = model.predict_proba(X_test_all)
y=Y_test

# generate additional metrics
recall = metrics.recall_score(y,y_pred, average='weighted')
precision = metrics.precision_score(y,y_pred, average='weighted')
accuracy = metrics.accuracy_score(y,y_pred)
F1 = metrics.f1_score(y,y_pred, average='weighted')
print("Sensitivity/Recall (TPR): ",recall)
print("Precision (PPV): ", precision)
print("Accuracy: ", accuracy)
print("F1:", F1)

#generate confusion matrix
conf_mat = confusion_matrix(y, y_pred)
print('Confusion matrix:\n', conf_mat)
   

In [ ]:
#use Kernel SHAP to explain test set predictions
explainer = shap.KernelExplainer(model.predict_proba, X_train_all, link="logit")
shap_values = explainer.shap_values(X_test_all, nsamples=100)

#plot the SHAP values for the Setosa output of the first instance
shap.force_plot(explainer.expected_value[0], shap_values[0][-2,:], X_test_all.iloc[-2,:], link="logit")

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X_test_all)

In [ ]:
# plot the SHAP values for the Setosa output of all instances
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test_all, link="logit")